# MNIST distributed training and batch transform

The SageMaker Python SDK helps you deploy your models for training and hosting in optimized, production-ready containers in SageMaker. The SageMaker Python SDK is easy to use, modular, extensible and compatible with TensorFlow and MXNet. This tutorial focuses on how to create a convolutional neural network model to train the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) using TensorFlow distributed training.

## Set up the environment

First, we'll just set up a few things needed for this example

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.session import Session

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_session.region_name

role = get_execution_role()

### Download the MNIST dataset

We'll now need to download the MNIST dataset, and upload it to a location in S3 after preparing for training.

In [ ]:
import utils
from tensorflow.contrib.learn.python.learn.datasets import mnist
import tensorflow as tf

data_sets = mnist.read_data_sets('data', dtype=tf.uint8, reshape=False, validation_size=5000)

utils.convert_to(data_sets.train, 'train', 'data')
utils.convert_to(data_sets.validation, 'validation', 'data')
utils.convert_to(data_sets.test, 'test', 'data')

### Upload the data
We use the ```sagemaker.Session.upload_data``` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use this later when we start the training job.

In [ ]:
inputs = sagemaker_session.upload_data(path='data', key_prefix='data/DEMO-mnist')

# Construct a script for distributed training 
Here is the full code for the network model:

In [ ]:
!cat 'mnist.py'

## Create a training job

In [ ]:
from sagemaker.tensorflow import TensorFlow

mnist_estimator = TensorFlow(entry_point='mnist.py',
                             role=role,
                             framework_version='1.11.0',
                             training_steps=1000, 
                             evaluation_steps=100,
                             train_instance_count=2,
                             train_instance_type='ml.c5.4xlarge')

mnist_estimator.fit(inputs)

The `fit()` method will create a training job in two ml.c4.xlarge instances. The logs above will show the instances doing training, evaluation, and incrementing the number of training steps. 

In the end of the training, the training job will generate a saved model for TF serving.

In [ ]:
model = mnist_estimator.create_model()
response = sagemaker_session.create_model(
    name=mnist_estimator.latest_training_job.name,
    role=mnist_estimator.role,
    container_defs = model.prepare_container_def('ml.c4.xlarge')
)

## Now, let's run inference from Lambda

Return to the workshop directions.